In [ ]:
!pip install -q google-adk

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set

# Movie Recommender Agent

Build an agent that helps users find movies to watch. Your agent will have these tools:

1. **search_movies** — Search for movies by genre, year, or keyword
2. **get_movie_rating** — Get ratings and reviews for a specific movie
3. **check_streaming** — Check which streaming platforms have a movie available

Tools return hardcoded/fake data — the goal is designing the **interface** between the model and your code.

## Setup

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

## TODO 1: Define your tools

Just write plain Python functions. The ADK infers the schema from:
- **Type hints** → parameter types
- **Docstring** → tool description

The first tool is done for you as an example. Implement the remaining two.

In [ ]:
# Example tool (complete)

def search_movies(query: str, genre: str) -> dict:
    """Search for movies matching a query and genre.

    Args:
        query: Search keywords (e.g. "space adventure", "romantic comedy").
        genre: Movie genre filter (e.g. "sci-fi", "comedy", "drama", "horror").

    Returns:
        dict: status and matching movies or error message.
    """
    return {
        "status": "success",
        "report": (
            f"Found 3 {genre} movies matching '{query}': "
            f"1) Interstellar (2014) — 8.7/10. "
            f"2) The Martian (2015) — 8.0/10. "
            f"3) Arrival (2016) — 7.9/10."
        ),
    }


def get_movie_rating(title: str) -> dict:
    """Get ratings and review summary for a specific movie.

    Args:
        title: The movie title (e.g. "Inception").

    Returns:
        dict: status and rating details or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return hardcoded ratings from multiple sources (IMDb, Rotten Tomatoes, etc.)
    pass


def check_streaming(title: str, country: str) -> dict:
    """Check which streaming platforms have a movie available.

    Args:
        title: The movie title (e.g. "Inception").
        country: Country code for regional availability (e.g. "US", "UK").

    Returns:
        dict: status and streaming availability or error message.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return a list of platforms where the movie is available (Netflix, Prime, etc.)
    pass

## TODO 2: Define your agent

In [ ]:
agent = Agent(
    model="gemini-3-flash-preview",
    name="movie_recommender",
    # TODO: Write an instruction that gives the agent a film-enthusiast persona.
    # Hint: describe how it should recommend movies, handle preferences, spoiler policy, etc.
    instruction="""You are a helpful movie recommender.

Replace this with your agent's persona and guidelines.
""",
    tools=[search_movies, get_movie_rating, check_streaming],
)

## Run your agent

Once you've filled in the TODOs above, run this cell to test your agent!

In [ ]:
runner = InMemoryRunner(agent=agent, app_name="movie_recommender")
runner.auto_create_session = True
user_content = types.Content(role="user", parts=[types.Part.from_text(text="I'm in the mood for a sci-fi movie about space. What do you recommend? And where can I stream it in the US?")])

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                final_text = part.text

print(final_text)

## Tips

- **Docstrings matter** — the ADK uses them as tool descriptions. Be specific about what each parameter means.
- **Fake data is fine** — the goal is designing the agent's interface, not building a real movie database.
- **Test edge cases** — what happens when a user asks about a movie your tools don't know about?

### Stretch Goals
- Add a `get_similar_movies` tool that finds movies similar to one the user liked
- Add a `get_movie_details` tool that returns cast, director, and plot summary
- Support mood-based recommendations (e.g. "I want something uplifting")